In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from scipy.special import expit
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude,longitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1757921916,2025-09-15 04:38,1757921400,2025-09-15 04:30,11.1,52.0,0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003,2.9,4.7,90,E,1021.0,30.15,0.0,0.0,100,75,11.1,52.0,11.8,53.2,11.7,53.1,10.7,51.3,10.0,6.0,0.0,4.6,7.5,"[{'date': '2025-09-15', 'date_epoch': 17578944..."


In [4]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-09-15,1757894400,"[{'time_epoch': 1757905200, 'time': '2025-09-1...",27.4,81.3,11.4,52.6,17.5,63.5,9.2,14.8,0.11,0.0,0.0,9.7,6.0,72,1,88,0,0,Possibilidade de chuva irregular,//cdn.weatherapi.com/weather/64x64/day/176.png,1063,2.1,06:14 AM,06:10 PM,01:51 AM,12:16 PM,Waning Crescent,44,0,0


In [5]:
hour_list = dados["forecast"]["forecastday"][0]["hour"]
df_hour = pd.json_normalize(hour_list)
df_hour


,time_epoch,time,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,wind_dir,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition.text,condition.icon,condition.code
0,1757905200,2025-09-15 00:00,12.7,54.8,0,4.5,7.2,79,E,1020.0,30.13,0.00,0.0,0.0,89,65,12.3,54.1,12.3,54.1,12.7,54.8,11.0,51.7,0,0,0,0,10.0,6.0,6.2,10.0,0.0,Nublado,//cdn.weatherapi.com/weather/64x64/night/119.png,1006
1,1757908800,2025-09-15 01:00,12.7,54.9,0,4.3,6.8,78,ENE,1020.0,30.12,0.00,0.0,0.0,89,55,12.4,54.4,12.4,54.4,12.7,54.9,11.0,51.8,0,0,0,0,10.0,6.0,5.9,9.5,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
2,1757912400,2025-09-15 02:00,12.4,54.4,0,3.6,5.8,81,E,1019.0,30.10,0.00,0.0,0.0,90,50,12.3,54.2,12.3,54.2,12.4,54.4,10.8,51.4,0,0,0,0,10.0,6.0,5.3,8.5,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
3,1757916000,2025-09-15 03:00,12.1,53.7,0,3.4,5.4,84,E,1019.0,30.10,0.00,0.0,0.0,92,61,12.0,53.6,12.0,53.6,12.1,53.7,10.8,51.5,0,0,0,0,10.0,6.0,5.1,8.3,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
4,1757919600,2025-09-15 04:00,11.1,52.0,0,2.9,4.7,90,E,1021.0,30.15,0.00,0.0,0.0,100,75,11.8,53.2,11.8,53.2,11.7,53.1,10.7,51.3,0,0,0,0,10.0,6.0,4.6,7.5,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
5,1757923200,2025-09-15 05:00,11.4,52.6,0,2.9,4.7,89,E,1020.0,30.12,0.00,0.0,0.0,95,70,11.5,52.6,11.5,52.6,11.4,52.6,10.6,51.1,0,0,0,0,2.0,1.0,4.8,7.7,0.0,Neblina,//cdn.weatherapi.com/weather/64x64/night/143.png,1030
6,1757926800,2025-09-15 06:00,12.1,53.8,0,2.7,4.3,90,E,1021.0,30.14,0.00,0.0,0.0,91,66,12.3,54.2,12.3,54.2,12.1,53.8,10.6,51.2,0,0,0,0,10.0,6.0,3.9,6.3,0.0,Nublado,//cdn.weatherapi.com/weather/64x64/night/119.png,1006
7,1757930400,2025-09-15 07:00,14.5,58.1,1,3.6,5.8,72,ENE,1021.0,30.14,0.00,0.0,0.0,79,19,14.7,58.4,14.7,58.4,14.5,58.1,11.0,51.7,0,0,0,0,10.0,6.0,4.1,6.6,0.2,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000
8,1757934000,2025-09-15 08:00,17.5,63.6,1,4.3,6.8,67,ENE,1020.0,30.12,0.00,0.0,0.0,65,4,17.6,63.6,17.6,63.6,17.6,63.6,11.0,51.7,0,0,0,0,10.0,6.0,4.9,7.9,1.1,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000
9,1757937600,2025-09-15 09:00,20.8,69.4,1,4.3,6.8,66,ENE,1019.0,30.10,0.00,0.0,0.0,53,4,20.8,69.4,20.8,69.4,20.8,69.4,10.9,51.7,0,0,0,0,10.0,6.0,4.9,7.9,3.1,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000


In [6]:
# Lista apenas com as colunas importantes
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
    "condition.text"
]

df_limpo = df_hour[colunas_importantes]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain,condition.text
17,2025-09-15 17:00,18.6,70,7,0.6,13.7,0,0,Sol
2,2025-09-15 02:00,12.3,90,50,0.0,5.8,0,0,Parcialmente nublado
10,2025-09-15 10:00,24.8,45,4,5.8,6.5,0,0,Sol
11,2025-09-15 11:00,25.7,40,6,8.1,7.2,0,0,Sol
12,2025-09-15 12:00,26.5,36,8,9.4,7.6,0,0,Sol


In [7]:
# Cria uma tabela associando estilos de roupa com dados do clima
df_tabela1 = pd.DataFrame({
    "estilos": ["casual", "esportivo", "social", "streetwear"],
    "data e hora": df_limpo["time"].values[:4],
    "temperatura": df_limpo["feelslike_c"].values[:4],
    "chance de chuva": df_limpo["chance_of_rain"].values[:4],
    "umidade": df_limpo["humidity"].values[:4],
    "uv": df_limpo["uv"].values[:4],
    "condição do dia": df_limpo ["condition.text"].values[:4]
})

df_tabela1

,estilos,data e hora,temperatura,chance de chuva,umidade,uv,condição do dia
0,casual,2025-09-15 00:00,12.3,0,89,0.0,Nublado
1,esportivo,2025-09-15 01:00,12.4,0,89,0.0,Parcialmente nublado
2,social,2025-09-15 02:00,12.3,0,90,0.0,Parcialmente nublado
3,streetwear,2025-09-15 03:00,12.0,0,92,0.0,Parcialmente nublado


In [8]:
#subdados para peças de roupas por estilo especifico

# subdados_roupas = {
#     "casual": {
#         "casual para calor": ["camisetas polo", "camisetas básicas de algodão", "camisas de manga curta"],
#         "casual para frio": ["suéter leve", "jaqueta jeans", "jaqueta de sarja"],
#         "casual meia estação": ["camisa xadrez", "camisa de botões leve", "cardigan fino"],
#     },
#     "esportivo": {
#         "esportivo para calor": ["camisetas dry-fit", "regatas, shorts esportivos", "bermudas de tecido leve"],
#         "esportivo para frio": ["moletom com capuz", "calça jogger", "calça de moletom"],
#         "esportivo meia estação": ["blusa térmica fina", "calça legging", "jaqueta leve de treino"],
#     },
#     "social": {
#         "social para calor": ["camisas de linho", "camisas de manga curta sociais", "calças de alfaiataria leve"],
#         "social para frio": ["blazer estruturado", "suéter de lã", "colete de alfaiataria"],
#         "social meia estação": ["trench coat leve", "camisa social slim fit", "calça social reta"],
#     },
#     "streetwear": {
#         "streetwear para calor": ["camisetas oversized", "camisas estampadas leves", "bermudas cargo"],
#         "streetwear para frio": ["hoodies com estampas", "jaquetas bomber", "jaquetas jeans oversized"],
#         "streetwear meia estação": ["jaqueta utilitária", "camisa xadrez oversized", "calça jogger estilizada"],
#     }
# }

In [9]:
print(df_tabela1.columns.tolist())

['estilos', 'data e hora', 'temperatura', 'chance de chuva', 'umidade', 'uv', 'condição do dia']


In [10]:
#transformando variáveis e em um número categorico 

df_tabela = pd.get_dummies(df[df_tabela1])
x = pd.concat([df_tabela1[["temperatura"]], df_tabela], axis=1)
y = df_tabela1["estilos"]


In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,4
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [13]:
# exemplo: 22°C, 70% umidade, UV 5, vento 10, sem chuva
novo_exemplo = pd.DataFrame([{
    "temperatura": 22,
    "chance_de_chuva": 0,
    "umidade": 70,
    "uv": 5
}])


In [14]:

# print("Recomendação:", clf.score(X_test, y_test))

In [15]:
# temperatura = [[df_tabela1]]  # exemplo: 22°C, 70% umidade, UV 5, vento 10, sem chuva
# print("Recomendação:", clf.predict(df_tabela1)[0])